#Fine-Tuning OpenAI Models

Copyright 2024 Denis Rothman

[OpenAI fine-tuning documentation](https://beta.openai.com/docs/guides/fine-tuning/)

Check the cost of fine-tuning your dataset on OpenAI before running the notebook.

Run this notebook cell by cell to:

1.Download and prepare the data   
2.Fine-tune a model   
3.Run a fine-tuned model

# Installing the environment


In [1]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)
#Comment this cell if you want to enter your key manually.
#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)

f = open("api_key.txt", "r")
API_KEY=f.readline()
f.close()

In [2]:
try:
  import openai
except:
  !pip install openai==1.42.0
  import openai

In [3]:
#(2) Enter your manually by
# replacing API_KEY by your key.
#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
!pip install jsonlines==4.0.0

In [5]:
!pip install datasets==2.20.0

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.5 MB/s eta 0:00:00
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 14.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.17
    Uninstalling multiprocess-0.70.17:
      Successfully uninstalled multiprocess-0.70.17
ERROR: pip's dependency resolver does not currently take into account all the pack

# 1.Preparing the dataset for fine-tuning

## 1.1.Downloading and displaying the dataset

In [6]:
# Import required libraries
from datasets import load_dataset
import pandas as pd

# Load the SciQ dataset from HuggingFace
dataset_view = load_dataset("sciq", split="train")

# Filter the dataset to include only questions with support and correct answer
filtered_dataset = dataset_view.filter(lambda x: x["support"] != "" and x["correct_answer"] != "")


# Print the number of questions with support
print("Number of questions with support: ", len(filtered_dataset))

/home/dk/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filter: 100%|██████████| 11679/11679 [00:00<00:00, 92343.68 examples/s]

Number of questions with support:  10481


In [7]:
# Convert the filtered dataset to a pandas DataFrame
df_view = pd.DataFrame(filtered_dataset)

# Columns to drop
columns_to_drop = ['distractor3', 'distractor1', 'distractor2']

# Dropping the columns from the DataFrame
df_view = df_view.drop(columns=columns_to_drop)

# Display the DataFrame
df_view.head()

,question,correct_answer,support
0,What type of organism is commonly used in prep...,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


## 1.2. Preparing the dataset for fine-tuning

In [9]:
import json
import jsonlines
import pandas as pd
from datasets import load_dataset

# Load and clean the dataset as previously described
dataset = load_dataset("sciq", split="train")
filtered_dataset = dataset.filter(lambda x: x["support"] != "" and x["correct_answer"] != "")

# Convert to DataFrame and clean
df = pd.DataFrame(filtered_dataset)
columns_to_drop = ['distractor3', 'distractor1', 'distractor2']
df = df.drop(columns=columns_to_drop)

# Prepare the data items for JSON lines file
items = []
for idx, row in df.iterrows():
    detailed_answer = row['correct_answer'] + " Explanation: " + row['support']
    items.append({
        "messages": [
            {"role": "system", "content": "Given a science question, provide the correct answer with a detailed explanation."},
            {"role": "user", "content": row['question']},
            {"role": "assistant", "content": detailed_answer}
        ]
    })

# Write to JSON lines file
with jsonlines.open('QA_prompts_and_completions.json', 'w') as writer:
    writer.write_all(items)

### Visualizing the JSON file

In [10]:
dfile="QA_prompts_and_completions.json"

In [12]:
import pandas as pd

# Load the data
df = pd.read_json(dfile, lines=True)
df
json_file = df.to_csv('QA_prompts_and_completions1.csv', index=False)

# 2.Fine-tuning the model



In [14]:
from openai import OpenAI
import jsonlines
client = OpenAI(api_key=API_KEY)
# Uploading the training file

result_file = client.files.create(
  file=open("QA_prompts_and_completions.json", "rb"),
  purpose="fine-tune"
)

print(result_file)
param_training_file_name = result_file.id
print(param_training_file_name)

# Creating the fine-tuning job

ft_job = client.fine_tuning.jobs.create(
  training_file=param_training_file_name,
  model="gpt-4o-mini-2024-07-18"
)

# Printing the fine-tuning job
print(ft_job)

FileObject(id='file-SuvWyYnNoshZKZuqZcE3op', bytes=8062970, created_at=1736018291, filename='QA_prompts_and_completions.json', object='file', purpose='fine-tune', status='processed', status_details=None)
file-SuvWyYnNoshZKZuqZcE3op
FineTuningJob(id='ftjob-JwXdWKAx4sYia7ciXpoGyykc', created_at=1736018293, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-XcM2BaYyk0NvFQSS4W505dsY', result_files=[], seed=545414696, status='validating_files', trained_tokens=None, training_file='file-SuvWyYnNoshZKZuqZcE3op', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})


## Monitoring the fine-tunes

In [22]:
import pandas as pd
from openai import OpenAI
client = OpenAI()
# Assume client is already set up and authenticated
response = client.fine_tuning.jobs.list(limit=3) # increase to include your history

# Initialize lists to store the extracted data
job_ids = []
created_ats = []
statuses = []
models = []
training_files = []
error_messages = []
fine_tuned_models = []  # List to store the fine-tuned model names

# Iterate over the jobs in the response
for job in response.data:
    job_ids.append(job.id)
    created_ats.append(job.created_at)
    statuses.append(job.status)
    models.append(job.model)
    training_files.append(job.training_file)
    error_message = job.error.message if job.error else None
    error_messages.append(error_message)

    # Append the fine-tuned model name
    fine_tuned_model = job.fine_tuned_model if hasattr(job, 'fine_tuned_model') else None
    fine_tuned_models.append(fine_tuned_model)

# Create a DataFrame
df = pd.DataFrame({
    'Job ID': job_ids,
    'Created At': created_ats,
    'Status': statuses,
    'Model': models,
    'Training File': training_files,
    'Error Message': error_messages,
    'Fine-Tuned Model': fine_tuned_models  # Include the fine-tuned model names
})

# Convert timestamps to readable format
df['Created At'] = pd.to_datetime(df['Created At'], unit='s')
df = df.sort_values(by='Created At', ascending=False)

# Display the DataFrame
df

,Job ID,Created At,Status,Model,Training File,Error Message,Fine-Tuned Model
0,ftjob-JwXdWKAx4sYia7ciXpoGyykc,2025-01-04 19:18:13,succeeded,gpt-4o-mini-2024-07-18,file-SuvWyYnNoshZKZuqZcE3op,None,ft:gpt-4o-mini-2024-07-18:personal::Am5LmIHd
1,ftjob-EJxfErp9JUf8hy8gVhgdwnU3,2025-01-04 19:17:39,cancelled,gpt-4o-mini-2024-07-18,file-1msR8bYDHw8QazheVvHRTL,None,None


### Make sure to obtain your fine-tune model here

If your OpenAI notifications are activated you should receive an email.

Otherwise run the "Monitoring the fine-tunes" cell above to check the status of your fine-tune job.

In [23]:
import pandas as pd

generation=False  # until the current model is fine-tuned
# Attempt to find the first non-empty Fine-Tuned Model
non_empty_models = df[df['Fine-Tuned Model'].notna() & (df['Fine-Tuned Model'] != '')]

if not non_empty_models.empty:
    first_non_empty_model = non_empty_models['Fine-Tuned Model'].iloc[0]
    print("The latest fine-tuned model is:", first_non_empty_model)
    generation=True
else:
    first_non_empty_model='None'
    print("No fine-tuned models found.")

The latest fine-tuned model is: ft:gpt-4o-mini-2024-07-18:personal::Am5LmIHd


*Note:* Only continue to Step 3, to use the fine-tuned model when your fine-tuned model is ready. If your OpenAI notifications is activiated, you will receive an email with the status of your fine-tunning job.

# 3.Using the fine-tuned OpenAI model

Note: The is a fine-tuning. As such, be patient!
Rune the `Monitoring the fine-tunes` cell and the f`irst_non_empty_model` cell from time to time.

If the fine-tunning succeeded and your model is ready, the name of your model will be `first_non_empty_model`

1.Go to the OpenAI Playground to test your model: https://platform.openai.com/playground

2.Check the metrics in the fine-tuning UI:
https://platform.openai.com/finetune/

3.Try the fined-tune model out in the cell below.

In [24]:
# Define the prompt
prompt = "What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?"

*Note:* Only run the following cell if your fine-tune job has succeeded and a fined-tuned model is found in the *Monitoring the fine-tunes"* section of *2.Fine-tuning the model.*

In [25]:
# Assume first_non_empty_model is defined above this snippet
if generation==True:
    response = client.chat.completions.create(
        model=first_non_empty_model,
        temperature=0.0,  # Adjust as needed for variability
        messages=[
            {"role": "system", "content": "Given a question, reply with a complete explanation for students."},
            {"role": "user", "content": prompt}
        ]
    )
else:
    print("Error: Model is None, cannot proceed with the API request.")

In [26]:
if generation==True:
  print(response)

ChatCompletion(id='chatcmpl-AmNdAa195AFFgGJQUSsbcoXi8SZBh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='coriolis effect Explanation: Global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere. This is due to the Coriolis effect.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1736093944, model='ft:gpt-4o-mini-2024-07-18:personal::Am5LmIHd', object='chat.completion', service_tier=None, system_fingerprint='fp_944baebe1a', usage=CompletionUsage(completion_tokens=42, prompt_tokens=51, total_tokens=93, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))


In [27]:
if (generation==True):
  # Access the response from the first choice
  response_text = response.choices[0].message.content
  # Print the response
  print(response_text)

coriolis effect Explanation: Global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere. This is due to the Coriolis effect.


In [28]:
import textwrap

if generation==True:
  wrapped_text = textwrap.fill(response_text.strip(), 60)
  print(wrapped_text)

coriolis effect Explanation: Global winds blow northeast to
southwest or the reverse in the northern hemisphere and
northwest to southeast or the reverse in the southern
hemisphere. This is due to the Coriolis effect.


[Consult OpenAI fine-tune documentation for more](https://platform.openai.com/docs/guides/fine-tuning)